In [16]:
import os
import cv2
import time
import numpy as np
from PIL import Image
from openvino.inference_engine import IECore
%matplotlib inline
from matplotlib import pyplot as plt
print('Imported Python modules successfully.')

Imported Python modules successfully.


In [3]:
# python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model masked_face_classifier.pb --input_shape [32,96,96,3]

In [2]:
# model IR files
model_xml = "FMIC/densenet201_non_rmfd_32_fix.xml"
model_bin = "FMIC/densenet201_non_rmfd_32_fix.bin"

# input image file
input_path = "./dog.jpg"
   
# CPU extension library to use
cpu_extension_path = os.path.expanduser("~")+"/inference_engine_samples/intel64/Release/lib/libcpu_extension.so"

# device to use
device = "CPU"

# number of top results to display
report_top_n = 10

# output labels 
labels_path = "FMIC/densenet201_non_rmfd_32_fix.txt"

print("Configuration parameters settings:"
     "\n\tmodel_xml=", model_xml,
      "\n\tmodel_bin=", model_bin,
      "\n\tinput_path=", input_path,
      "\n\tdevice=", device, 
      "\n\tlabels_path=", labels_path, 
      "\n\treport_top_n=", report_top_n)

Configuration parameters settings:
	model_xml= FMIC/densenet201_non_rmfd_32_fix.xml 
	model_bin= FMIC/densenet201_non_rmfd_32_fix.bin 
	input_path= ./dog.jpg 
	device= CPU 
	labels_path= FMIC/densenet201_non_rmfd_32_fix.txt 
	report_top_n= 10


In [3]:
# create Inference Engine instance
ie = IECore()
print("An Inference Engine object has been created")

An Inference Engine object has been created


In [4]:
# load network from IR files
net = ie.read_network(model=model_xml, weights=model_bin)
print("Loaded model IR files [",model_bin,"] and [", model_xml, "]\n")

# check to make sure that the plugin has support for all layers in the loaded model
supported_layers = ie.query_network(net,device)
not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
if len(not_supported_layers) != 0:
    print("ERROR: Following layers are not supported by the plugin for specified",
          " device {}:\n {}".format(device, ', '.join(not_supported_layers)))
    assert 0 == 1, "ERROR: Missing support for all layers in the model," \
               + " cannot continue."

# check to make sue that the model's input and output are what is expected
assert len(net.inputs.keys()) == 1, \
    "ERROR: This sample supports only single input topologies"
assert len(net.outputs) == 1, \
    "ERROR: This sample supports only single output topologies"
print("SUCCESS: Model IR files have been loaded and verified")

Loaded model IR files [ FMIC/densenet201_non_rmfd_32_fix.bin ] and [ FMIC/densenet201_non_rmfd_32_fix.xml ]

SUCCESS: Model IR files have been loaded and verified


In [5]:
exec_net = ie.load_network(network=net, num_requests=2, device_name=device)

# store name of input and output blobs
input_blob = next(iter(net.inputs))
output_blob = next(iter(net.outputs))

# read the input's dimensions: n=batch size, c=number of channels, h=height, w=width
n, c, h, w = net.inputs[input_blob].shape
print("Loaded model into Inference Engine for device:", device, 
      "\nModel input dimensions: n=",n,", c=",c,", h=",h,", w=",w)

Loaded model into Inference Engine for device: CPU 
Model input dimensions: n= 32 , c= 3 , h= 32 , w= 32


In [6]:
labels_map = None
# if labels points to a label mapping file, then load the file into labels_map
print(labels_path)
if os.path.isfile(labels_path):
    with open(labels_path, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
    print("Loaded label mapping file [",labels_path,"]")
else:
    print("No label mapping file has been loaded, only numbers will be used",
          " for detected object labels")

FMIC/densenet201_non_rmfd_32_fix.txt
Loaded label mapping file [ FMIC/densenet201_non_rmfd_32_fix.txt ]


In [10]:
labels_map = ['Wearing Mask', 'Not Wearing Mask']

In [32]:
res_img = 32

# img = cv2.imread('./test_dataset/1/0_0_0.jpg')
img = cv2.imread('./test_dataset/0/1_0_aidai_0001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (res_img,res_img), interpolation=cv2.INTER_CUBIC)
img = img/255
img = img.transpose((2, 0, 1)) 
img = np.expand_dims(img, axis=0)

In [33]:
img.shape

(1, 3, 32, 32)

In [34]:
res = exec_net.infer(inputs={input_blob: img})

In [35]:
res

{'argmax/Squeeze': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)}

In [ ]:
np.expan